# CoT (Chain of Thought)

In [6]:
from dotenv import load_dotenv
import openai

load_dotenv()
client = openai.OpenAI()

### 일반 프롬프트 vs Zero-Shot CoT

In [7]:
# 일반 프롬프트
prompt = '123 x 31 + 0.123 x 0.0012는 얼마인가요?'

response = client.chat.completions.create(
    model='gpt-4',
    messages=[{'role': 'user', 'content': prompt}]
)

In [8]:
print(response.choices[0].message.content)

3813.0000001476입니다.


In [9]:
# CoT 프롬프트
prompt = '단계별로 계산하세요.: 123 x 31 + 0.123 x 0.0012는 얼마인가요?'

cot_response = client.chat.completions.create(
    model='gpt-4',
    messages=[{'role': 'user', 'content': prompt}]
)

In [10]:
print(cot_response.choices[0].message.content)

먼저 숫자를 곱합니다:

123 x 31 = 3813
0.123 x 0.0012 = 0.0001476

그 다음 두 결과를 더합니다:

3813 + 0.0001476 = 3813.0001476

따라서 최종 결과는 3813.0001476입니다.


### Few-Shot CoT

In [11]:
few_shot_cot = '''
Q: 123 x 31은 얼마인가요?
A:
1. 123을 분해합니다: 123 = 100 + 20 + 3
2. 각 항을 31과 곱합니다:
    - 100 x 31 = 3100
    - 20 x 31 = 620
    - 3 x 31 = 93
3. 이제 이 결과들을 더합니다:
    - 3100 + 620 + 93
4. 계산을 진행하면:
    - 3100 + 620 = 3720
    - 3720 + 93 = 3813
따라서 123 x 31 = 3813 입니다.

Q: 789 x 56 은 얼마인가요?
A:
'''

cot_response = client.chat.completions.create(
    model='gpt-4',
    messages=[{'role': 'user', 'content': few_shot_cot}]
)

In [12]:
print(cot_response.choices[0].message.content)

1. 789를 분해합니다: 789 = 700 + 80 + 9
2. 각 항을 56과 곱합니다:
    - 700 x 56 = 39200
    - 80 x 56 = 4480
    - 9 x 56 = 504
3. 이제 이 결과들을 더합니다:
    - 39200 + 4480 + 504
4. 계산을 진행하면:
    - 39200 + 4480 = 43680
    - 43680 + 504 = 44184
따라서 789 x 56 = 44184 입니다.


### 논리 문제 해결

In [16]:
prompt = '''
단계적으로 생각하면서 대답해주세요:
앨리스(Alice)는 밥(Bob)보다 나이가 많다.
밥(Bob)은 찰리()보다 나이가 많다.
데이비드(David)는 앨리스(Alice)보다 어리지만, 에밀리(Emily) 보다는 많다.
찰리(Charlie)는 프랭크(Frank)보다 많다.
에밀리(Emily)는 찰리(Charlie)보다 나이가 많지만, 밥(Bob)보다는 어리다.
프랭크(Frank)는 데이비드(David)보다 어리다.
그렇다면 가장 어린 사람은 누구인가요?
'''

In [17]:
response = client.chat.completions.create(
    model='gpt-4',
    messages=[{'role': 'user', 'content': prompt}]
)

In [19]:
print(response.choices[0].message.content)

프랭크(Frank)가 가장 어립니다.

이를 파악하기 위해 각 문장을 분석해 보겠습니다:
1. "앨리스(Alice)는 밥(Bob)보다 나이가 많다"라는 문장은 앨리스가 밥보다 나이가 많다는 것을 알려줍니다.
2. "밥(Bob)은 찰리(Charlie)보다 나이가 많다"라는 문장은 밥이 찰리보다 나이가 많다는 것을 보여줍니다. 따라서 밥은 앨리스보다 어리고, 찰리는 밥보다 어립니다.
3. "데이비드(David)는 앨리스(Alice)보다 어리지만, 에밀리(Emily) 보다는 많다"라는 문장은 데이비드가 앨리스보다 어리지만 에밀리보다 늙다는 것을 나타냅니다.
4. "찰리(Charlie)는 프랭크(Frank)보다 많다"라는 문장은 찰리가 프랭크보다 늙었다는 것을 보여줍니다. 그러므로 프랭크는 밥보다 어리고 찰리보다 어립니다.
5. "에밀리(Emily)는 찰리(Charlie)보다 나이가 많지만, 밥(Bob)보다는 어리다"라는 문장은 에밀리가 찰리보다 늙었지만, 밥보다 어리다는 것을 알려줍니다.
6. "프랭크(Frank)는 데이비드(David)보다 어리다"라는 문장은 프랭크가 데이비드보다 어리다는 것을 보여주며, 이는 프랭크가 가장 어린 사람이라는 것을 확실하게 해줍니다.
따라서, 가장 어린 사람은 프랭크(Frank)입니다.
